In [1]:
!pwd
!ls

# In SageMaker Studio Code Editor, the following is the output:
# /home/sagemaker-user/histology-image-analysis/sagemaker-inference
# mhist-predict.ipynb  src  test_locally.py

# In a SageMaker Notebook Instance, the following is the output:
# /home/ec2-user/SageMaker/histology-image-analysis/sagemaker-inference
# mhist-predict.ipynb  model  src  test_locally.py

/home/sagemaker-user/histology-image-analysis/sagemaker-inference
MHIST_ViT_v13_dynamo_model.onnx  inference.py	      requirements.txt
__pycache__			 mhist-predict.ipynb  test_locally.py


In [2]:
# Test inference locally
!pip install -U -q -r requirements.txt
%run test_locally.py

Output: {"logit": 3.948834180831909, "predicted_class": "SSA", "probability": 0.9810874185378591}


In [3]:
!tar -czvf model.tar.gz MHIST_ViT_v13_dynamo_model.onnx inference.py requirements.txt

MHIST_ViT_v13_dynamo_model.onnx
inference.py
requirements.txt


In [4]:
# Upload model files to SageMaker default bucket
# !pip install -U sagemaker
import boto3
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
dest = sagemaker_session.upload_data(path='model.tar.gz', bucket=bucket, key_prefix='mhist-vit-model')

print(f"Model files uploaded to: {dest}")
# Model files uploaded to: s3://sagemaker-us-west-1-851725529671/mhist-vit-model/model.tar.gz

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Model files uploaded to: s3://sagemaker-us-west-1-851725529671/mhist-vit-model/model.tar.gz


In [9]:
import sagemaker
from sagemaker.pytorch import PyTorchModel
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role() # arn:aws:iam::851725529671:role/SageMakerEx
model_data = 's3://sagemaker-us-west-1-851725529671/mhist-vit-model/model.tar.gz'

model = PyTorchModel(
    model_data=model_data,
    role=role,
    entry_point='inference.py',
    source_dir='.',
    framework_version='2.3.0',
    py_version='py311',
    dependencies=['requirements.txt']
)
predictor = model.deploy(
    instance_type='ml.m5.xlarge',
    initial_instance_count=1
)
response = predictor.predict({
    'bucket': 'mhist-streamlit-app',
    'key': 'images/original/MHIST_aah.png'
})
predictor.delete_endpoint()
response

-----!

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary and could not load the entire response body. See https://us-west-1.console.aws.amazon.com/cloudwatch/home?region=us-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-2024-07-17-16-21-47-068 in account 851725529671 for more information.